# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
#Load the csv exported in Part I to a DataFrame
city_data = ("../output_data/cities.csv")
city_data_df = pd.read_csv(city_data)
city_data_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Angoche,-16.23,39.91,69.75,79,27,9.40,MZ,1594674273
1,Beyneu,45.32,55.20,88.59,21,17,11.83,KZ,1594674242
2,Hombourg-Haut,49.12,6.77,64.99,48,83,3.36,FR,1594674371
3,Hobart,-42.88,147.33,42.80,87,75,4.70,AU,1594674016
4,Atuona,-9.80,-139.03,79.54,77,24,19.35,PF,1594673992


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
fig = gmaps.figure(zoom_level=2,center=(20,40))
locations = city_data_df[["Lat","Lng"]]

# city_data_df = city_data_df.dropna()
humidity = city_data_df["Humidity"]

In [15]:
heatmap = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity = 10, point_radius=1)
fig.add_layer(heatmap)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
cities_narrow = city_data_df.loc[(city_data_df["Max_Temp"]>70) & (city_data_df["Max_Temp"]<80)&(city_data_df["Wind_Speed"]<10)&(city_data_df["Cloudiness"]==0),:].tail(10)
cities_narrow = cities_narrow.dropna(how="any")    
cities_narrow                                                                                 

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
275,Purwodadi Grobogan,-7.09,110.92,73.87,89,0,4.05,ID,1594674508
277,Kumul,42.80,93.45,72.63,37,0,8.48,CN,1594674260
323,Carinhanha,-14.30,-43.77,77.67,30,0,4.03,BR,1594674527
336,Coruripe,-10.13,-36.18,74.91,82,0,5.37,BR,1594674532
455,Masallātah,32.62,14.00,71.53,86,0,3.56,LY,1594674582
471,Caravelas,-17.71,-39.25,72.68,86,0,4.70,BR,1594674589
534,Bukama,-9.20,25.85,73.08,32,0,1.90,CD,1594674620
545,Nador,35.17,-2.93,75.20,88,0,3.36,MA,1594674626
549,Sidi Ali,36.10,0.42,72.18,77,0,9.15,DZ,1594674282
565,Zliten,32.47,14.57,75.63,83,0,5.17,LY,1594674432


In [17]:
len(cities_narrow)

10

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotel_df = cities_narrow.copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
275,Purwodadi Grobogan,-7.09,110.92,73.87,89,0,4.05,ID,1594674508,
277,Kumul,42.80,93.45,72.63,37,0,8.48,CN,1594674260,
323,Carinhanha,-14.30,-43.77,77.67,30,0,4.03,BR,1594674527,
336,Coruripe,-10.13,-36.18,74.91,82,0,5.37,BR,1594674532,
455,Masallātah,32.62,14.00,71.53,86,0,3.56,LY,1594674582,


In [19]:
 # Use the search Hotel Name
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# params dictionary to update each iteration

params = {"radius": 5000,
          "type": "lodging",
          "keyword": "hotel",
          "key": g_key}

# Use the lat/lng we recovered to identify Hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # make request and print url
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        print("Missing Hotel information.....skip")
        hotel_df.loc[index, "Hotel Name"] = "n/a"
        pass

Missing Hotel information.....skip
Missing Hotel information.....skip
Missing Hotel information.....skip


In [20]:
# Hotel data inforamtion
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
275,Purwodadi Grobogan,-7.09,110.92,73.87,89,0,4.05,ID,1594674508,Kyriad Grand Master Hotel Purwodadi
277,Kumul,42.80,93.45,72.63,37,0,8.48,CN,1594674260,7 Days Premium
323,Carinhanha,-14.30,-43.77,77.67,30,0,4.03,BR,1594674527,Hotel União
336,Coruripe,-10.13,-36.18,74.91,82,0,5.37,BR,1594674532,Pousada Pontal Sunset
455,Masallātah,32.62,14.00,71.53,86,0,3.56,LY,1594674582,n/a
471,Caravelas,-17.71,-39.25,72.68,86,0,4.70,BR,1594674589,Mar Doce de Abrolhos
534,Bukama,-9.20,25.85,73.08,32,0,1.90,CD,1594674620,n/a
545,Nador,35.17,-2.93,75.20,88,0,3.36,MA,1594674626,"Hotel Marchica Lagoon Resort, Nador Morocco"
549,Sidi Ali,36.10,0.42,72.18,77,0,9.15,DZ,1594674282,n/a
565,Zliten,32.47,14.57,75.63,83,0,5.17,LY,1594674432,فندق الهادي العاتي


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, 
              row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
markers_lon_lat = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(markers_lon_lat,info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(markers)
fig.add_layer(heatmap)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))